In [1]:
import psycopg2

# create db connnection
def create_connection():
    try:
        connnection = psycopg2.connect(
            dbname="musicstreamingdb",
            user="postgres",
            password="postgrespass",
            host="localhost"
        )
        return connnection
    except Exception as e:
        print('Error connecting to the db:', e)
        return None

    connection.create_connection()
    cursor = connection.cursor()

In [2]:
# Function to execute SQL query and print the result
def execute_sql(query, fetch_result=False):
    connection = create_connection()
    if connection:
        try:
            cursor = connection.cursor()
            cursor.execute(query)
            
            if fetch_result:
                result = cursor.fetchall()  # Fetch all results for SELECT queries
                return result  # Return result to be printed by the caller
            
            connection.commit()  # Commit changes 
            print("Query executed successfully.")
        except Exception as e:
            print("Error executing SQL query:", e)
        finally:
            cursor.close()
            connection.close()

# Function to run a query and print results for SELECT queries
def execute_and_print_query(query):
    result = execute_sql(query, fetch_result=True)
    if result:
        for row in result:
            print(row)
    else:
        print("No data returned.")

In [ ]:
'''
    #8 An example of a view that has a hard-coded criteria, by which the content of the view may change upon changing the hard-coded value
'''
# This query will display the spotify song's with a popularity score above 80
QUERY = """
CREATE OR REPLACE VIEW HighPopularitySpotifySongs AS 
SELECT S.title, S.artist, S.genre, SS.spotify_popularity_score AS popularity_scr
FROM Songs S
JOIN SpotifySongs SS ON S.song_id = SS.song_id
WHERE SS.spotify_popularity_score > 80;
"""
execute_and_print_query(QUERY)

In [ ]:
'''
    #9 Two queries that demonstrate the overlap and covering constraints.
'''
# Overlap:
QUERY1 = """
SELECT S.title FROM Songs S
JOIN SpotifySongs SS ON S.song_id = SS.song_id
JOIN LastfmSongs LFMS ON S.song_id = LFMS.song_id;
"""

# Covering:
QUERY2= """ 
SELECT S.title 
FROM Songs S 
WHERE NOT EXISTS (
    SELECT 1 FROM SpotifySongs AS SS 
    WHERE SS.song_id = S.song_id
)
AND NOT EXISTS (
    SELECT 1 FROM LastfmSongs AS LFMS
    WHERE LFMS.song_id = S.song_id
);
"""
execute_and_print_query(QUERY1)
execute_and_print_query(QUERY2)

In [ ]:
'''
    #10 Two implementations of the division operator using 
         a) a regular nested query using NOT IN 
         b) a correlated nested query using NOT EXISTS and EXCEPT 
'''
# Both questies ensure that a song exists in both Spotify and Sound Cloud
# Using NOT IN:
QUERY1 = """
SELECT S.title FROM Songs S
WHERE S.song_id NOT IN (
    SELECT SS.song_id FROM Songs S
    LEFT JOIN SpotifySongs SS ON S.song_id = SS.song_id
    WHERE SS.song_id IS NULL
)
AND S.song_id NOT IN (
    SELECT LFMS.song_id FROM Songs S 
    LEFT JOIN LastfmSongs LFMS ON S.song_id = LFMS.song_id
    WHERE LFMS.song_id IS NULL
);
"""

# Using NOT EXISTS and EXCEPT:
QUERY2 = """ 
SELECT S.title FROM Songs S
WHERE NOT EXISTS (
    SELECT 1 FROM (
        SELECT song_id FROM SpotifySongs
        EXCEPT 
        SELECT song_id FROM LastfmSongs
    ) AS SpotifyButNotInSoundCloud
    WHERE S.song_id = SpotifyButNotInSoundCloud.song_id
);
"""
execute_and_print_query(QUERY1)
execute_and_print_query(QUERY2)